## Fig. 7: comparison of the size distributions

In [ ]:
%run header.py

In [ ]:
d      = np.load(opacity.get_datafile('default_opacities_smooth.npz'))
a      = d['a']
gsca   = d['g']
lam    = d['lam']
k_abs  = d['k_abs']
k_sca  = d['k_sca']

In [ ]:
r      = c.au.cgs.value                # position in the disk in cm
TEMP   = [100, 200., 500]              # temperature in K
SIG_G  = [100, 200., 500]              # gas surface density [g/cm^2]
d2g    = 0.01                          # dust-to-gas ratio
rho_s  = 1.6686                        # material density [g/cm^3]
M_star = c.M_sun.cgs.value             # stellar mass [g]
V_FRAG = [100., 1000, 2000]            # fragmentation velocity [cm/s]
ALPHA  = [1e-4, 1e-3, 1e-2]            # turbulence parameter
m      = 4 * np.pi / 3 * rho_s * a**3  # mass grid

params = aux.permutate_fiducial([TEMP, SIG_G, V_FRAG, ALPHA])
labels = [r'T = {:g} K', '$\Sigma_\mathrm{{g}} = {:g}$ g/cm$^2$', '$v_\mathrm{{frag}}$ = {:g} m/s', r'$\alpha = 10^{{{:g}}}$']


f, axs = plt.subplots(4, 1, figsize=(3.5, 4 * 3.5 * 0.575), sharex=True, sharey=True)
axs = axs.flat


for i_param in range(len(params[0])):
    ax = axs[i_param]
    
    params_slice = [params[-1], params[2 * i_param], params[2 * i_param + 1]]
    
    params_slice = sorted(params_slice, key = lambda o: o[i_param])
    
    for values in params_slice:
        
        T, sig_g, v_frag, alpha = values

        # create the size distribution, FIRST: the fortran code

        fit, a_01, a_12, a_l, a_p, a_r, a_sett = opacity.distribution(1.8, T, alpha, sig_g, sig_g * d2g, rho_s, m, a, M_star, r, v_frag)

        # SECOND: a simplified version. Need to convert to bin-integrated values.

        fit2, a_f = opacity.get_B11S_fit(T, a, r=r, sigma_g=sig_g, d2g=d2g, rho_s=rho_s, M_star=M_star, v_frag=v_frag, alpha=alpha)
        fit2 = fit2 * np.diff(np.hstack((a[0]**2 / a[1], a)))
        fit2 = fit2 / fit2.sum() * sig_g * d2g
        
        # plotting
        
        val = values[i_param]
        if i_param==2:
            val /=100
        if i_param==3:
            val = np.log10(val)
        
        l1, = ax.loglog(a, fit, label=labels[i_param].format(val))
        l2, = ax.loglog(a, fit2, '--', c=l1.get_color())
        
    ax.loglog([], [], 'k--', label='B11S')
    ax.legend(fontsize='x-small', loc=2)
    ax.set_ylabel('$\sigma$ [g cm$^{-2}$]')
        
ax.set_xlim(1e-5, 2e2)
ax.set_ylim(6e-5, 2e-1)
ax.set_xlabel('particle size [cm]')

f.subplots_adjust(
    bottom=0.05, top=0.98,
    left=0.14, right=0.97,
    hspace=0.05
    )

f.savefig('figures/fig7_distri_panels.pdf')